In [1]:
from datasets import load_dataset
swebench = load_dataset('princeton-nlp/SWE-bench', cache_dir='./cache')

In [4]:
print(swebench)

# Inspect individual splits
for split in swebench:
    print(f"\n--- {split.upper()} SPLIT ---")
    print(swebench[split])

# Analyze feature types and data
for split in swebench:
    print(f"\nFeature types in {split} split:")
    print(swebench[split].features)
    print(f"\nSample data from {split} split:")
    print(swebench[split][0])  # Display the first record in the split

# Get number of examples per split
split_sizes = {split: swebench[split].num_rows for split in swebench}
print("\nDataset split sizes:", split_sizes)

# Check class distribution for classification tasks (FAIL_TO_PASS, PASS_TO_PASS flags)
for label in ['FAIL_TO_PASS', 'PASS_TO_PASS']:
    for split in swebench:
        label_counts = swebench[split].to_pandas()[label].value_counts()
        print(f"\nDistribution of {label} in {split} split:")
        print(label_counts)

DatasetDict({
    dev: Dataset({
        features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
        num_rows: 225
    })
    test: Dataset({
        features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
        num_rows: 2294
    })
    train: Dataset({
        features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
        num_rows: 19008
    })
})

--- DEV SPLIT ---
Dataset({
    features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
    

In [2]:
# use as pandas and iloc 0
first_row = swebench["dev"].to_pandas().iloc[0]
first_row

repo                                                        sqlfluff/sqlfluff
instance_id                                           sqlfluff__sqlfluff-4764
base_commit                          a820c139ccbe6d1865d73c4a459945cd69899f8f
patch                       diff --git a/src/sqlfluff/cli/commands.py b/sr...
test_patch                  diff --git a/test/cli/commands_test.py b/test/...
problem_statement           Enable quiet mode/no-verbose in CLI for use in...
hints_text                                                                   
created_at                                               2023-04-16T14:24:42Z
version                                                                   1.4
FAIL_TO_PASS                ["test/cli/commands_test.py::test__cli__fix_mu...
PASS_TO_PASS                ["test/cli/commands_test.py::test__cli__comman...
environment_setup_commit             d19de0ecd16d298f9e3bfb91da122734c40c01e5
Name: 0, dtype: object

# Run Llama Peft

In [3]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-Python-hf")
model = PeftModel.from_pretrained(base_model, "princeton-nlp/SWE-Llama-7b-peft")

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

C:\Users\kosti\anaconda3\envs\ai_env\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kosti\.cache\huggingface\hub\models--codellama--CodeLlama-7b-Python-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

C:\Users\kosti\anaconda3\envs\ai_env\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kosti\.cache\huggingface\hub\models--princeton-nlp--SWE-Llama-7b-peft. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


adapter_model.bin:   0%|          | 0.00/67.2M [00:00<?, ?B/s]

In [6]:
from transformers import AutoTokenizer

# Φόρτωση tokenizer
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Python-hf")

# Δείγμα εισόδου
input_text = "def print_hello_function(x):"

# Tokenization
inputs = tokenizer(input_text, return_tensors="pt")

# Χρήση GPU αν είναι διαθέσιμο
inputs = {key: val.to("cpu") for key, val in inputs.items()}
model.to("cpu")

# Δημιουργία αποτελέσματος
outputs = model.generate(**inputs, max_length=50)

# Αποκωδικοποίηση του αποτελέσματος
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)